## Machine Learning 프로젝트 수행을 위한 코드 구조화

- ML project를 위해서 사용하는 템플릿 코드를 만듭니다.

1. **필요한 라이브러리와 데이터를 불러옵니다.**


2. **EDA를 수행합니다.** 이 때 EDA의 목적은 풀어야하는 문제를 위해서 수행됩니다.


3. **전처리를 수행합니다.** 이 때 중요한건 **feature engineering**을 어떻게 하느냐 입니다.


4. **데이터 분할을 합니다.** 이 때 train data와 test data 간의 분포 차이가 없는지 확인합니다.


5. **학습을 진행합니다.** 어떤 모델을 사용하여 학습할지 정합니다. 성능이 잘 나오는 GBM을 추천합니다.


6. **hyper-parameter tuning을 수행합니다.** 원하는 목표 성능이 나올 때 까지 진행합니다. 검증 단계를 통해 지속적으로 **overfitting이 되지 않게 주의**하세요.


7. **최종 테스트를 진행합니다.** 데이터 분석 대회 포맷에 맞는 submission 파일을 만들어서 성능을 확인해보세요.

## 1. 라이브러리, 데이터 불러오기

In [51]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

In [96]:
# 데이터를 불러옵니다.
base_path = '../data/'
train = pd.read_csv(base_path + 'train.csv')
test = pd.read_csv(base_path + 'test.csv')
submission = pd.read_csv(base_path + 'sample_submission.csv')
print(train.shape, test.shape, submission.shape)

(101763, 23) (67842, 22) (67842, 2)


## 2. EDA

- 데이터에서 찾아야 하는 기초적인 내용들을 확인합니다.


- class imbalance, target distribution, outlier, correlation을 확인합니다.

In [53]:
train.columns

Index(['id', 'loc', 'v(g)', 'ev(g)', 'iv(g)', 'n', 'v', 'l', 'd', 'i', 'e',
       'b', 't', 'lOCode', 'lOComment', 'lOBlank', 'locCodeAndComment',
       'uniq_Op', 'uniq_Opnd', 'total_Op', 'total_Opnd', 'branchCount',
       'defects'],
      dtype='object')

### 3. 전처리

#### 결측치 처리

### 4. 학습 데이터 분할

In [54]:
# 첫번째 테스트용으로 사용하고, 실제 학습시에는 K-Fold CV를 사용합니다.
from sklearn.model_selection import train_test_split

X = train.drop(columns=['defects'])
y = train.defects

# for OOF-prediction split 5% of data as validation dataset.
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=61, stratify=y)

In [55]:
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)
print(y_train.mean(), y_val.mean())

(81410, 22) (81410,) (20353, 22) (20353,)
0.2266429185603734 0.22664963396059548


### 5. 학습 및 평가

In [69]:
# 간단하게 LightGBM 테스트
# 적당한 hyper-parameter 조합을 두었습니다. (항상 best는 아닙니다. 예시입니다.)
model = XGBClassifier(
    booster='gbtree',
    objective='binary:logistic',
    max_depth=6,
    learning_rate=0.1,
    n_estimators=100,
    n_jobs=-1,
    random_state=61
)

In [70]:
print("\nFitting LightGBM...")
model.fit(X_train, y_train)


Fitting LightGBM...


XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=61, ...)

In [71]:
# metric은 그때마다 맞게 바꿔줘야 합니다.
evaluation_metric = roc_auc_score

In [72]:
print("Prediction")
pred_train = model.predict(X_train)
pred_val = model.predict(X_val)


train_score = evaluation_metric(y_train, pred_train)
val_score = evaluation_metric(y_val, pred_val)

print("Train Score : %.4f" % train_score)
print("Validation Score : %.4f" % val_score)

Prediction
Train Score : 0.6810
Validation Score : 0.6668


### 6. Hyper-parameter Tuning

In [76]:
def optimizer(trial, X, y, K):
    # 조절할 hyper-parameter 조합을 적어줍니다.
    max_depth = trial.suggest_int('max_depth', 5, 15)
    colsample_bynode = trial.suggest_float('colsample_bynode', 0.5, 0.8)
    reg_lambda = trial.suggest_float('reg_lambda', 0.5, 5.0)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)

    # 원하는 모델을 지정합니다, optuna는 시간이 오래걸리기 때문에 저는 보통 RF로 일단 테스트를 해본 뒤에 LGBM을 사용합니다.
    model = XGBClassifier(
        max_depth=max_depth,
        colsample_bynode=colsample_bynode,
        reg_lambda=reg_lambda,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        random_state=61,
        eval_metric=evaluation_metric,
    )

    # K-Fold Cross validation을 구현합니다.
    folds = StratifiedKFold(n_splits=K, random_state=61, shuffle=True)
    losses = []

    for train_idx, val_idx in folds.split(X, y):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]

        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]

        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        loss = evaluation_metric(y_val, preds)
        losses.append(loss)


    # K-Fold의 평균 loss값을 돌려줍니다.
    return np.mean(losses)

In [77]:
K = 5   # Kfold 수
opt_func = partial(optimizer, X=X, y=y, K=K)

study = optuna.create_study(direction="maximize") # 최소/최대 어느 방향의 최적값을 구할 건지.
study.optimize(opt_func, n_trials=50)

[I 2023-10-12 14:56:42,775] A new study created in memory with name: no-name-71bd27b7-1a5f-4e84-963d-0a7ba3ac0092
[I 2023-10-12 14:57:20,620] Trial 0 finished with value: 0.6570820404811688 and parameters: {'max_depth': 10, 'colsample_bynode': 0.6527806841449566, 'reg_lambda': 4.123427390080673, 'n_estimators': 118, 'learning_rate': 0.22144245045640804}. Best is trial 0 with value: 0.6570820404811688.
[I 2023-10-12 15:03:38,066] Trial 1 finished with value: 0.6524280341374044 and parameters: {'max_depth': 13, 'colsample_bynode': 0.5807836985163611, 'reg_lambda': 0.5803673820441073, 'n_estimators': 966, 'learning_rate': 0.21702513139757007}. Best is trial 0 with value: 0.6570820404811688.
[I 2023-10-12 15:04:55,717] Trial 2 finished with value: 0.661503931030469 and parameters: {'max_depth': 14, 'colsample_bynode': 0.7517793825465595, 'reg_lambda': 1.4908876418319543, 'n_estimators': 104, 'learning_rate': 0.04302162358741328}. Best is trial 2 with value: 0.661503931030469.
[I 2023-10-12

In [78]:
# optuna가 시도했던 모든 실험 관련 데이터
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_colsample_bynode,params_learning_rate,params_max_depth,params_n_estimators,params_reg_lambda,state
0,0,0.657082,2023-10-12 14:56:42.777631,2023-10-12 14:57:20.619982,0 days 00:00:37.842351,0.652781,0.221442,10,118,4.123427,COMPLETE
1,1,0.652428,2023-10-12 14:57:20.623046,2023-10-12 15:03:38.065175,0 days 00:06:17.442129,0.580784,0.217025,13,966,0.580367,COMPLETE
2,2,0.661504,2023-10-12 15:03:38.067174,2023-10-12 15:04:55.716025,0 days 00:01:17.648851,0.751779,0.043022,14,104,1.490888,COMPLETE
3,3,0.662883,2023-10-12 15:04:55.718025,2023-10-12 15:06:14.099310,0 days 00:01:18.381285,0.632793,0.036863,10,203,4.565398,COMPLETE
4,4,0.660059,2023-10-12 15:06:14.101394,2023-10-12 15:07:11.762639,0 days 00:00:57.661245,0.680095,0.136110,5,353,4.198561,COMPLETE
5,5,0.655964,2023-10-12 15:07:11.764781,2023-10-12 15:08:08.467280,0 days 00:00:56.702499,0.723493,0.251323,11,128,2.107829,COMPLETE
6,6,0.657083,2023-10-12 15:08:08.468265,2023-10-12 15:12:22.803178,0 days 00:04:14.334913,0.738288,0.115602,15,407,4.058388,COMPLETE
7,7,0.658763,2023-10-12 15:12:22.805187,2023-10-12 15:19:06.232013,0 days 00:06:43.426826,0.777936,0.035375,12,784,3.760789,COMPLETE
8,8,0.653591,2023-10-12 15:19:06.233012,2023-10-12 15:23:11.328612,0 days 00:04:05.095600,0.766420,0.247819,10,621,3.897971,COMPLETE
9,9,0.664886,2023-10-12 15:23:11.332062,2023-10-12 15:23:22.271343,0 days 00:00:10.939281,0.717500,0.196213,6,50,2.174700,COMPLETE


In [79]:
print("Best Score: %.4f" % study.best_value) # best score 출력
print("Best params: ", study.best_trial.params) # best score일 때의 하이퍼파라미터들

Best Score: 0.6672
Best params:  {'max_depth': 5, 'colsample_bynode': 0.6456616045114356, 'reg_lambda': 1.3644494601047512, 'n_estimators': 117, 'learning_rate': 0.04334999942547478}


In [80]:
# 실험 기록 시각화
optuna.visualization.plot_optimization_history(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [81]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

### 7. 테스트 및 제출 파일 생성

In [88]:
# Make KFold OOF prediction
def oof_preds(best_model):

    # make KFold
    folds = StratifiedKFold(n_splits=K, random_state=42, shuffle=True)
    final_preds = []
    losses = []
    # fitting with best_model
    for i, (train_idx, val_idx) in enumerate(folds.split(X, y)):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]
        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]

        print(f"========== Fold {i+1} ==========")
        best_model.fit(X_train, y_train)
        preds = best_model.predict_proba(X_val)[:, 1]
        test_preds = best_model.predict_proba(test)[:, 1]
        final_preds.append(test_preds)
        loss = evaluation_metric(y_val, preds)

        losses.append(loss)

    avg_loss = np.mean(losses)
    print(f"Loss : {avg_loss:.4f}")
    return final_preds

In [89]:
test.info() # 결측치 없음.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67842 entries, 0 to 67841
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 67842 non-null  int64  
 1   loc                67842 non-null  float64
 2   v(g)               67842 non-null  float64
 3   ev(g)              67842 non-null  float64
 4   iv(g)              67842 non-null  float64
 5   n                  67842 non-null  float64
 6   v                  67842 non-null  float64
 7   l                  67842 non-null  float64
 8   d                  67842 non-null  float64
 9   i                  67842 non-null  float64
 10  e                  67842 non-null  float64
 11  b                  67842 non-null  float64
 12  t                  67842 non-null  float64
 13  lOCode             67842 non-null  int64  
 14  lOComment          67842 non-null  int64  
 15  lOBlank            67842 non-null  int64  
 16  locCodeAndComment  678

In [91]:
## X_test 만들기 : 앞서했던 전처리를 동일하게 적용해주면 됨.


In [90]:
best_params = study.best_trial.params

# define best model
best_model = XGBClassifier(**best_params,
                           random_state=61)

# model finalization : 가장 일반적으로 좋은 예측 성능을 냈던 모델로, 전체 데이터 트레이닝.

preds = oof_preds(best_model)
preds = np.mean(preds, axis=0)
preds

========== Fold 1 ==========
========== Fold 2 ==========
========== Fold 3 ==========
========== Fold 4 ==========
========== Fold 5 ==========
Loss : 0.7919


array([0.26644674, 0.23265156, 0.6457157 , ..., 0.20588079, 0.11557309,
       0.7869202 ], dtype=float32)

In [97]:
submission['defects'] = preds

In [99]:
submission.to_csv(base_path+"submission_xgboost_kfold.csv", index=False)